In [2]:
from main.models import Zipcode, BlockGroup, Neighborhood, Listing
from django.contrib.gis.geos import (
    GEOSGeometry, 
    Polygon, 
    MultiPolygon,
    WKBReader, 
    WKBWriter,
    Point
)
from django.contrib.gis.gdal import DataSource
import re


## ZCTA Import

In [77]:
# Open US Zip Code Tabulation Areas shapefile
ds = DataSource('../res/cb_2015_us_zcta510_500k.kml')
layer = ds[0] # There's only 1 layer in this file

In [81]:
# Create and save zipcode objects
for feature in layer:
    digits = re.search("ZCTA5CE10<\/th>\s*<td>\s*(\d+)\s*<\/td>", feature.get('Description')).group(1)
    geom = GEOSGeometry(feature.geom.json, srid=4326)
    geom = WKBReader().read(wkb=WKBWriter(dim=2).write(geom)) # Coerce 3D input geometry to 2D
    zipcode = Zipcode(zipcode=digits)
    if geom.geom_type == 'MultiPolygon':
        zipcode.mpoly = geom
    else:
        zipcode.mpoly = MultiPolygon(geom)
    zipcode.save()

In [9]:
# For each LA zipcode, show the neighborhoods it overlaps
la_zipcodes = set([
    zipcode
    for n in Neighborhood.objects.all()
    for zipcode in Zipcode.objects.filter(mpoly__intersects=n.mpoly)
])

for zipcode in set(la_zipcodes):
    print(zipcode.zipcode, Neighborhood.objects.filter(mpoly__intersects=zipcode.mpoly), '\n')

90293 <QuerySet [<Neighborhood: Del Rey>, <Neighborhood: El Segundo>, <Neighborhood: Marina del Rey>, <Neighborhood: Playa Vista>, <Neighborhood: Playa del Rey>, <Neighborhood: Venice>, <Neighborhood: Westchester>]> 

91775 <QuerySet [<Neighborhood: Alhambra>, <Neighborhood: Arcadia>, <Neighborhood: East Pasadena>, <Neighborhood: East San Gabriel>, <Neighborhood: San Gabriel>, <Neighborhood: San Marino>, <Neighborhood: Temple City>]> 

90292 <QuerySet [<Neighborhood: Culver City>, <Neighborhood: Del Rey>, <Neighborhood: Marina del Rey>, <Neighborhood: Playa del Rey>, <Neighborhood: Venice>]> 

93563 <QuerySet [<Neighborhood: Angeles Crest>, <Neighborhood: Southeast Antelope Valley>]> 

91316 <QuerySet [<Neighborhood: Brentwood>, <Neighborhood: Encino>, <Neighborhood: Pacific Palisades>, <Neighborhood: Reseda>, <Neighborhood: Sepulveda Basin>, <Neighborhood: Tarzana>]> 

90043 <QuerySet [<Neighborhood: Baldwin Hills/Crenshaw>, <Neighborhood: Hyde Park>, <Neighborhood: Inglewood>, <Neigh

## Block Group Import

In [3]:
# Open Los Angeles County Block Groups shapefile
ds = DataSource('../res/cb_2015_06_bg_500k.kml')
layer = ds[0] # There's only 1 layer in this file

In [5]:
# Create and save BlockGroup objects
for feature in layer:
    geoid = re.search("GEOID<\/th>\s*<td>(\d{12})<\/td>", feature.get('Description')).group(1)
    geom = GEOSGeometry(feature.geom.json, srid=4326)
    geom = WKBReader().read(wkb=WKBWriter(dim=2).write(geom)) # Coerce 3D input geometry to 2D
    block_group = BlockGroup(geoid=geoid)
    if geom.geom_type == 'MultiPolygon':
        block_group.mpoly = geom
    else:
        block_group.mpoly = MultiPolygon(geom)
    block_group.save()

In [11]:
# Testing/exploration below


for i, bg in enumerate(BlockGroup.objects.all()):
    if i > 500:
        break
    print(bg.geoid, Neighborhood.objects.filter(mpoly__intersects=bg.mpoly), '\n')

060411122021 <QuerySet []> 

060750127001 <QuerySet []> 

060030100002 <QuerySet []> 

060490003002 <QuerySet []> 

060930004001 <QuerySet []> 

060730082002 <QuerySet []> 

060375702042 <QuerySet [<Neighborhood: Long Beach>]> 

060530110003 <QuerySet []> 

060530122001 <QuerySet []> 

060230115003 <QuerySet []> 

060371392002 <QuerySet [<Neighborhood: Encino>]> 

060250112011 <QuerySet []> 

060750331001 <QuerySet []> 

060374804002 <QuerySet [<Neighborhood: Alhambra>, <Neighborhood: San Marino>, <Neighborhood: South Pasadena>]> 

060375405022 <QuerySet [<Neighborhood: Compton>, <Neighborhood: Lynwood>]> 

060590876025 <QuerySet []> 

060190032022 <QuerySet []> 

060377008021 <QuerySet [<Neighborhood: Beverly Grove>, <Neighborhood: Beverly Hills>]> 

060014279005 <QuerySet []> 

060730101072 <QuerySet []> 

060375424011 <QuerySet [<Neighborhood: Compton>]> 

060371210101 <QuerySet [<Neighborhood: Sun Valley>]> 

060371317011 <QuerySet [<Neighborhood: Reseda>, <Neighborhood: Winnetka>]

In [5]:
hms = Point(y=33.8810, x=-118.1795)
BlockGroup.objects.filter(mpoly__contains=hms)
Neighborhood.objects.filter(mpoly__contains=hms)

<QuerySet [<Neighborhood: Long Beach>]>

In [6]:
Zipcode.objects.filter(mpoly__contains=hms)

<QuerySet [<Zipcode: 90805>]>